# Implementación de Algoritmo SVM

In [42]:
# -*- coding: utf-8 -*-
"""
Created on Fri May 10 11:49:17 2024

@author: ProfL3302
"""

import pandas as pd
import numpy as np
import re
from sklearn import preprocessing
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.svm import SVC
from sklearn.model_selection import cross_val_score, GridSearchCV
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold, LeaveOneOut, StratifiedKFold
from sklearn.metrics import classification_report, accuracy_score, make_scorer

pd.set_option('display.max_rows',None)
pd.set_option('display.max_columns',None)

def extractValues(file_path):
    extracted_values=[]
    with open(file_path,'r') as file:
        for line in file:
            matches=re.findall(r'\((.*?)\)',line)
            if matches:
                extracted_values.extend(matches)
    # print(extracted_values)
    return extracted_values

def checkEmpty(df):
    missing=df.isnull().sum()
    # print(missing)

def imputation(df):
    df=df.fillna(df.mode().iloc[0])
    print("verificando")
    checkEmpty(df)
    # print(df.info())
    # print(df.describe())
    return df


def encoding(df):
    #['age', 'sex', 'cp', 'trestbps', 'chol', 'fbs', 'restecg', 'thalach', 'exang', 'oldpeak', 'slope', 'ca', 'thal', 'num']
    X=df.iloc[:,:-1]
    listOrd=['sex','fbs','exang']
    listOhe=['cp','restecg','slope','thal']
    ppOrd=preprocessing.OrdinalEncoder()
    X[listOrd]=ppOrd.fit_transform(X[listOrd])
    ohe = OneHotEncoder(drop='first')
    ohe_output = ohe.fit_transform(X[listOhe]).toarray()
    ohe_feature_names=ohe.get_feature_names_out(listOhe)
    ohe_df=pd.DataFrame(ohe_output,columns=ohe_feature_names)
    X.drop(columns=listOhe,inplace=True)
    X_encoded=pd.concat([X,ohe_df],axis=1)
    print(X_encoded.head())
    return X_encoded

def svcCV(X,y,nVC):
    svc=SVC(C=1,kernel='rbf',gamma='auto',probability=True)
    if nVC == 1:
        #HOLD OUT
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
        svc.fit(X_train,y_train)
        print("Score Hold Out",svc.score(X_test,y_test))
        #completar para el conjunto de test
        svc.fit(X_test,y_test)
        print("Score Test",svc.score(X_test,y_test))
    elif nVC ==2 :
        kf = KFold(n_splits=7, shuffle=True, random_state=42)
        nested= cross_val_score(svc,X,y,cv=kf,scoring=make_scorer(Classification_report_with_acc))
        print(nested)
        print(classification_report(originalClass,predClass))
    
    elif nVC ==3:
        loocv = LeaveOneOut()
        nested= cross_val_score(svc,X,y,cv=loocv,scoring=make_scorer(Classification_report_with_acc))
        print(nested)
        print(classification_report(originalClass,predClass))
    
    elif nVC ==4:
        strCV = StratifiedKFold(n_splits=10, shuffle=True, random_state=1)
        nested= cross_val_score(svc,X,y,cv=strCV,scoring=make_scorer(Classification_report_with_acc))
        print(nested)
        print(classification_report(originalClass,predClass))
    

def load(fileR,namesCol):
    f=pd.read_csv(fileR,sep=',',names=namesCol)
    #print(f.head())
    df=pd.DataFrame(f)
    dfOrig=df
    y=df['num']
    #print(df.head())
    df.replace('?',np.nan,inplace=True)
    checkEmpty(df)
    #print(df.info())
    dfImp=imputation(df)
    
    dfOhe=encoding(dfImp)

    colScale=['age', 'trestbps', 'chol', 'thalach']
    scaler=StandardScaler()
    scaledCol=scaler.fit_transform(dfOhe[colScale]) 
    dfOhe[colScale]=scaledCol

    y_binary=y.apply(lambda x:1 if x>0 else 0)
    # svm(dfOhe,y_binary)
    # svc_grid_search(dfOhe,y_binary)

    typeCV=4
    svcCV(dfOhe,y_binary,typeCV)
    

def Classification_report_with_acc(y_true, y_pred):
    originalClass.extend(y_true)
    predClass.extend(y_pred)
    acc=accuracy_score(y_true, y_pred)
    return acc # return accuracy score

def svm(X,y):
    svc=SVC(C=1,kernel='rbf',gamma='auto',probability=True)
    cv_scores=cross_val_score(svc,X,y,cv=5)
    print("cv_scores",cv_scores)
    print("mean cv scores", cv_scores.mean())

def svc_grid_search(X,y): #colocar parametros con los que se han trabajado
    param_grid = {'C': [0.1, 1, 10, 100], 'gamma': ['scale','auto'], 'kernel': ['linear','rbf','sigmoid'], # 'degree': [1,2,3,4,5]} 
    }
    svc=SVC()
    grid_search= GridSearchCV(estimator=svc,param_grid=param_grid,cv=10)
    grid_search.fit(X,y)

    print("Hiperparametros ",grid_search.best_params_)
    print("Mejor Score ",grid_search.best_score_)

originalClass =[]
predClass = []
    


In [43]:
def main():
        f=r"D:\DATOS RECUPERADOS\UNIVERSIDAD DE LIMA\11. Last Dance\Machine Learning\Scripts de Clase\SEMANA 6\processed.cleveland.data"
        fileName = r"D:\DATOS RECUPERADOS\UNIVERSIDAD DE LIMA\11. Last Dance\Machine Learning\Scripts de Clase\SEMANA 6\clevelandNames.txt"
        namesCol=extractValues(fileName)
        load(f,namesCol)

main()


verificando
    age  sex  trestbps   chol  fbs  thalach  exang  oldpeak   ca  cp_2.0  \
0  63.0  1.0     145.0  233.0  1.0    150.0    0.0      2.3  0.0     0.0   
1  67.0  1.0     160.0  286.0  0.0    108.0    1.0      1.5  3.0     0.0   
2  67.0  1.0     120.0  229.0  0.0    129.0    1.0      2.6  2.0     0.0   
3  37.0  1.0     130.0  250.0  0.0    187.0    0.0      3.5  0.0     0.0   
4  41.0  0.0     130.0  204.0  0.0    172.0    0.0      1.4  0.0     1.0   

   cp_3.0  cp_4.0  restecg_1.0  restecg_2.0  slope_2.0  slope_3.0  thal_6.0  \
0     0.0     0.0          0.0          1.0        0.0        1.0       1.0   
1     0.0     1.0          0.0          1.0        1.0        0.0       0.0   
2     0.0     1.0          0.0          1.0        1.0        0.0       0.0   
3     1.0     0.0          0.0          0.0        0.0        1.0       0.0   
4     0.0     0.0          0.0          1.0        0.0        0.0       0.0   

   thal_7.0  
0       0.0  
1       0.0  
2       1.0  
